In [1]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import quantfreedom as qf
from plotly.subplots import make_subplots
from quantfreedom._typing import pdFrame
from quantfreedom.plotting.simple_plots import plot_results_candles_and_chart

In [2]:
price_data = qf.generate_candles(
    number_of_candles=200,
    plot_candles=False,
    seed=14,
)
price_data = pd.read_hdf('../tests/data/prices.hd5')
price_data = price_data[['BTCUSDT', 'EOSUSD']].join(pd.DataFrame())
rsi_data = qf.from_talib(
    func_name="rsi",
    price_data=price_data,
    timeperiod=[15,30],
    plot_results=False,
    # price="low",
)

In [15]:
def is_divergence_downtrend(
    want_to_evaluate: pdFrame,
    lookback: int,
    price_data: pdFrame = None,
    is_rising: float = 0,
):
    if isinstance(price_data, pdFrame):
        # returning this array in a dataframe
        final_eval_array = np.zeros_like(want_to_evaluate, dtype=np.bool_)

        # Ploting arrays
        piv_low_graph = np.full_like(want_to_evaluate, np.nan)
        divergence_index = np.full_like(want_to_evaluate, -1, dtype=np.int_)
        eval_line_graph = np.full_like(want_to_evaluate, np.nan)
        candle_line_graph = np.full_like(want_to_evaluate, np.nan)

        # vars
        col = 0
        symbols = list(price_data.columns.levels[0])

        for symbol in symbols:
            price_open_values = price_data[symbol]["open"].values
            price_high_values = price_data[symbol]["high"].values
            price_low_values = price_data[symbol]["low"].values
            price_close_values = price_data[symbol]["close"].values

            temp_want_to_eval = want_to_evaluate[symbol].values
            for values in temp_want_to_eval.T:
                piv_low = price_low_values[0]
                line_counter = 0
                for curr_candle in range(1, values.size):
                    temp_candle_low = price_low_values[curr_candle]
                    # if (
                    #     temp_candle_low < piv_low
                    #     and price_close_values[curr_candle]
                    #     < price_open_values[curr_candle]
                    # ):
                    for lb in range(1, lookback):
                        lb_index = curr_candle - lb
                        if lb_index > 0:    
                            if (
                                values[curr_candle] > values[lb_index] + is_rising
                                and price_close_values[lb_index]
                                < price_open_values[lb_index]
                            ):
                                final_eval_array[curr_candle, col] = True

                                # plotly graphing stuff
                                candle_line_graph[line_counter, col] = price_low_values[
                                    lb_index
                                ] - (price_low_values[lb_index] * 0.001)
                                candle_line_graph[
                                    line_counter + 1, col
                                ] = temp_candle_low - (temp_candle_low * 0.001)

                                eval_line_graph[line_counter, col] = values[
                                    lb_index
                                ] - (values[lb_index] * 0.001)
                                eval_line_graph[line_counter + 1, col] = values[
                                    curr_candle
                                ] - (values[curr_candle] * 0.001)

                                divergence_index[line_counter, col] = lb_index
                                divergence_index[line_counter + 1, col] = curr_candle

                                line_counter += 2

                                break
                        else:
                            break
                        # piv_low = temp_candle_low
                        # piv_low_graph[curr_candle, col] = temp_candle_low - (temp_candle_low * 0.001)
                    else:
                        pass
                col += 1
    # keep some type of counter that checks if we have gone 20 candles without a new low

    plot_index = price_data.index
    eval_line_graph = eval_line_graph[:line_counter, 0]
    candle_line_graph = candle_line_graph[:line_counter, 0]
    divergence_index = plot_index[divergence_index[:line_counter, -1].tolist()]

    fig = make_subplots(
        rows=2,
        cols=1,
        row_heights=[0.7, 0.3],
        shared_xaxes=True,
        vertical_spacing=0.02,
    )
    fig.append_trace(
        go.Candlestick(
            x=plot_index,
            open=price_open_values,
            high=price_high_values,
            low=price_low_values,
            close=price_close_values,
        ),
        row=1,
        col=1,
    )

    # fig.add_scatter(
    #     x=plot_index,
    #     y=piv_low_graph[:, 0],
    #     mode="markers",
    #     marker=dict(size=4, color="MediumPurple"),
    #     name="candle pivot",
    #     row=1,
    #     col=1,
    # )

    fig.append_trace(
        go.Scatter(
            x=plot_index,
            y=want_to_evaluate.values[:, 0],
            name="RSI",
        ),
        row=2,
        col=1,
    )

    start = 0
    end = 2
    for x in range(eval_line_graph.size):
        fig.add_scatter(
            x=divergence_index[start:end],
            y=candle_line_graph[start:end],
            mode="markers+lines",
            marker=dict(size=10, symbol="arrow-up"),
            name="Candle Line Div",
            row=1,
            col=1,
            showlegend=False,
        )
        fig.add_scatter(
            x=divergence_index[start:end],
            y=eval_line_graph[start:end],
            mode="markers+lines",
            marker=dict(size=10, symbol="arrow-up"),
            name="RSI div",
            row=2,
            col=1,
            showlegend=False,
        )
        start = end
        end += 2
    fig.update_layout(
        xaxis_rangeslider_visible=False,
        height=800,
    )
    fig.show()
    return pd.DataFrame(final_eval_array)

In [17]:
is_divergence_downtrend(
    want_to_evaluate=rsi_data,
    price_data=price_data,
    lookback=20,
    is_rising=0,
)


IndexError: index 9408 is out of bounds for axis 0 with size 9408